## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [155]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [156]:
# Load the dataset
debar = pd.read_csv('/Users/williamhubert/Documents/GitHub/QSS_Public/QSS20HW/code/pset2/pset2_inputdata/debar.csv')
debar.head()
# View the head of the dataset

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [157]:
# Print the total number of rows in the dataframe
print('Total rows:', len(debar))
# Print the number of unique employer names
print('Unique employers:', debar['Name'].nunique())

Total rows: 114
Unique employers: 98


## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [158]:
#A
debar['is_repeated'] = debar.duplicated('Name', keep = False)

# B
print(debar[debar['is_repeated'] == True])
# C
mult_debar = debar[debar['is_repeated'] == True]
print(mult_debar.head())
print(mult_debar.shape)
mult_debar

                                        Name       City, State  \
6                    Annabella Land & Cattle     Annabella, UT   
7                        Autumn Hill Orchard        Groton, MA   
8      Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
11                     Loewen Harvesting LLC   Brownsville, TX   
12               Rollo Farm Labor Contractor         Miami, FL   
14                             Sharon Mathis        Tifton, GA   
15                                 SRT Farms        Morton, TX   
16                               Mark Duncan     Roosevelt, UT   
17          Maple Ridge Custom Services, LLC     Altheimer, AK   
18                                 F&W Farms       Ingalls, KS   
19                        Cisco Produce Inc.         Cairo, GA   
21   Old Tree Farms/Verpaalen Custom Service         Volga, SD   
24               Rollo Farm Labor Contractor         Miami, FL   
25                     Loewen Harvesting LLC    Brownfield, TX   
28     Cad

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
14,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
15,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
16,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
17,"Maple Ridge Custom Services, LLC","Altheimer, AK",Failure to respond to audit (partial response),2 years,11/16/2014,11/15/2016,True
18,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True


Maple Ridge Custom Services, LLC           2
Loewen Harvesting LLC                      2
Old Tree Farms/Verpaalen Custom Service    2
Dove Creek Farms                           2
Cisco Produce Inc.                         2
                                          ..
Glenda Harrod Farms                        1
Chris Robinson                             1
Haiti/USA Workforce, LLC                   1
Leonard Smith Farms                        1
Delia Rojas                                1
Name: Name, Length: 98, dtype: int64

## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [159]:
# Assign a violation number to each row of the same employer based on 'Start date'
debar['violation_num'] = debar.sort_values('Start date').groupby(['Name', 'City, State']).cumcount() + 1
# Filter out only those rows that are marked as repeated 
mult_debar = debar[debar['is_repeated']]
# Pivot the dataframe to wide format
mult_debar_wide = mult_debar.pivot(index=['Name', 'City, State'], columns='violation_num', values='Start date')
# Rename columns
mult_debar_wide.columns = ['start_date_viol'+str(i) for i in range(1, len(mult_debar_wide.columns)+1)]
# Reset index
mult_debar_wide.reset_index(inplace=True)
# Print the head of the data
print(mult_debar_wide.head())
# Print the shape of the data
print(mult_debar_wide.shape)

debar


                                    Name      City, State start_date_viol1  \
0                Annabella Land & Cattle    Annabella, UT         5/9/2014   
1                Annabella Land & Cattle  Annabella, Utah         5/9/2014   
2                    Autumn Hill Orchard       Groton, MA         7/6/2014   
3  Caddo Creek Ranch, dba Paradise Ranch        Caddo, TX        7/20/2014   
4  Caddo Creek Ranch, dba Paradise Ranch     Caddo, Texas        7/20/2014   

  start_date_viol2  
0              NaN  
1              NaN  
2         7/6/2014  
3              NaN  
4              NaN  
(20, 4)


,Name,"City, State",Violation,Duration,Start date,End date,is_repeated,violation_num
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016,False,1.0
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016,False,1.0
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015,False,1.0
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,False,1.0
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,False,1.0
...,...,...,...,...,...,...,...,...
109,Dove Creek Farms,"Mount Vernon, TX",Failure to Respond to Audit Request,2 years,2/9/2018,2/8/2020,True,1.0
110,Jesus Ledesma,"Mulberry, FL",Failure to Respond to Audit Request,2 years,2/8/18,2/8/20,False,1.0
111,Turner Farms,"Healy, KS",Failure to comply with the employer's obligati...,7 months,7/17/19,2/10/20,True,2.0
112,B & R Harvesting and Paul Cruz (individual),"Collins, Georgia",WHD Debarment,3 years,4/9/17,4/9/20,False,1.0


## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [160]:
# A
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']

# B
# Merge is_dup back to mult_debar
merged_debar = pd.merge(mult_debar, mult_debar_wide[['Name', 'City, State', 'is_dup']], on=['Name', 'City, State'], how='left')

merged_debar.set_index(mult_debar.index, inplace=True)

# Filter rows
viol1_rows = merged_debar[(merged_debar['is_dup'] == True) & (mult_debar['violation_num'] == 1)]
other_rows = merged_debar[merged_debar['is_dup'] == False]
unchecked_rows = debar[debar["is_repeated"]== False]

# Concatenate the dataframes
debar_clean = pd.concat([viol1_rows, other_rows, unchecked_rows], axis=0)

# Remove the is_repeated column
debar_clean.drop(columns='is_repeated', inplace=True)
debar.drop(columns='is_repeated', inplace=True)

# Print the shape and # of unique employer names
print(debar_clean.shape)
print('Unique employers:', debar_clean['Name'].nunique())



(105, 8)
Unique employers: 98


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [161]:
# Load the dataset
jobs_banned = pd.read_csv('/Users/williamhubert/Documents/GitHub/QSS_Public/QSS20HW/code/pset2/pset2_inputdata/jobs.csv')
# View the head of the dataset
print(jobs_banned.head())



          CASE_NUMBER                           CASE_STATUS  \
0  H-300-20199-721302      Determination Issued - Withdrawn   
1  H-300-20231-773906  Determination Issued - Certification   
2  H-300-20231-774123  Determination Issued - Certification   
3  H-300-20231-774151  Determination Issued - Certification   
4  H-300-20231-774508  Determination Issued - Certification   

             RECEIVED_DATE            DECISION_DATE  \
0  2020-07-17 14:50:40.840  2020-10-01 00:00:00.000   
1  2020-08-20 10:38:15.620  2020-10-01 00:00:00.000   
2  2020-08-24 15:33:14.340  2020-10-01 00:00:00.000   
3  2020-08-21 12:08:09.760  2020-10-01 00:00:00.000   
4  2020-08-20 10:17:34.530  2020-10-01 00:00:00.000   

  TYPE_OF_EMPLOYER_APPLICATION H2A_LABOR_CONTRACTOR NATURE_OF_TEMPORARY_NEED  \
0          Individual Employer                    N                 Seasonal   
1          Association - Agent                    N                 Seasonal   
2          Individual Employer                    

##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [ ]:
jobs_banned

In [162]:
# print(jobs_banned)
# print(debar_clean)
matches = jobs_banned.merge(debar_clean, left_on='EMPLOYER_NAME', right_on='Name', how='inner')

matches

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,violation_num,is_dup
0,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,Rafael Barajas,NaN,...,https://seasonaljobs.dol.gov/,7,Rafael Barajas,"Sebring, Florida",Non-payment of certification fee,1 year,9/23/2016,9/22/2017,1.0,NaN


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [176]:
jobs_banned['EMPLOYER_NAME'] = [name.upper() for name in jobs_banned['EMPLOYER_NAME']]
debar_clean['Name'] = [name.upper() for name in debar_clean['Name']]


In [177]:
print(jobs_banned['EMPLOYER_NAME'].sample(15))
print(debar_clean['Name'].sample(15))

# C is completed in part A

65                         TEMP.LABOR, LLC
555                         LA ALIANZA, LP
1076             WESTERN RANGE ASSOCIATION
2131           WALLACE FARMS A PARTNERSHIP
1025                    TRIPLE E FARMS, LP
1490          SANTANA RANCH (ROBERT DIXON)
2                        MICHAEL RUDEBUSCH
975                   STOKLEY NURSERY, LLC
1132             WESTERN RANGE ASSOCIATION
2386    PHILLIPS BROTHERS FARM & LIVESTOCK
2427                   INGALLS HONEY, INC.
2130              DENNIS FARMS PARTNERSHIP
1164                      HE SHEEP COMPANY
2485                   SHORELINE FARMS, LC
573                OASIS HARVESTING, INC. 
Name: EMPLOYER_NAME, dtype: object
86                   DAVID C. MARTINEZ
77               MYRKA MIREYA CARDENAS
3                ANTON FERTILIZER INC.
17    MAPLE RIDGE CUSTOM SERVICES, LLC
47                        PETER PETERS
81                         OLSON FARMS
49       CITY PINESTRAW AND HARVESTING
70            JOHN & NETA LEOPKY FARMS
43      

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [183]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [184]:
# Define the regular expression pattern
pattern = r'(INC|LLC|CO)\.'

# Function to perform the replacement
def replace_dot(match):
    return match.group(1) + ' '

# Apply the regular expression substitution
jobs_banned_no_punctuation = [re.sub(pattern, replace_dot, employer) for employer in jobs_banned.EMPLOYER_NAME]
debar_clean_no_punctuation = [re.sub(pattern, replace_dot, employer) for employer in debar_clean.Name]

example_1 = [re.sub(pattern, replace_dot, pos_example_1)]
example_2 = [re.sub(pattern, replace_dot, pos_example_2)]
neg_example = [re.sub(pattern, replace_dot, neg_example)]

print(example_1, example_2, neg_example)

['CISCO PRODUCE INC '] ['AVOYELLES HONEY CO , LLC'] ['E.V. RANCH LLP']


### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [191]:
# Define the regular expression pattern
pattern = r'(INC|LLC|CO)\.'

# Function to perform the replacement
def replace_dot(match):
    return match.group(1) + ' '

debar_clean["name_clean"] = [re.sub(pattern, replace_dot, employer) for employer in debar_clean.Name]

jobs_banned["name_clean"] = [re.sub(pattern, replace_dot, employer) for employer in jobs_banned.EMPLOYER_NAME]

debar_clean["changed_name"] = ~(debar_clean["name_clean"] == debar_clean["Name"])
debar_changed = debar_clean[debar_clean.changed_name]
debar_changed

jobs_banned["changed_name"] = ~(jobs_banned["name_clean"] == jobs_banned["EMPLOYER_NAME"])
jobs_banned_no_punctuation = jobs_banned[jobs_banned.changed_name]
jobs_banned_no_punctuation

,Name,"City, State",Violation,Duration,Start date,End date,violation_num,is_dup,name_clean,changed_name
19,CISCO PRODUCE INC.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,1.0,False,CISCO PRODUCE INC,True
56,CISCO PRODUCE INC.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,2.0,False,CISCO PRODUCE INC,True
3,ANTON FERTILIZER INC.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,1.0,NaN,ANTON FERTILIZER INC,True
4,"GREAT PLAINS FLUID SERVICE, INC.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,1.0,NaN,"GREAT PLAINS FLUID SERVICE, INC",True
5,PROMAX INC.,"Whitewater, KS",Failure to Hire U.S. workers,2 years,5/15/2014,5/14/2016,1.0,NaN,PROMAX INC,True
13,REIMER'S INC.,"Lakin, KS",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,1.0,NaN,REIMER'S INC,True
23,REIMER’S INC.,"Lakin, Kansas",Impeding the Audit Process – Non- Response,2 years,8/23/2014,8/22/2016,1.0,NaN,REIMER’S INC,True
33,GERONIMO SHEEP CO.,"Vernal, UT",WHD Debarment,3 years,6/23/2014,6/22/2017,1.0,NaN,GERONIMO SHEEP CO,True
46,ALTENDORF TRANSPORT INC.,"Minto, ND",Impeding the Audit Process – Non- Response,2 years,4/24/2015,4/23/2017,1.0,NaN,ALTENDORF TRANSPORT INC,True
72,"SAXTONS RIVER ORCHARDS, INC.","Saxtons River, Vermont",Failure to respond to audit request,2 years,7/9/2016,7/8/2018,1.0,NaN,"SAXTONS RIVER ORCHARDS, INC",True


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,changed_name
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"DUNSON HARVESTING, INC.","Dunson Harvesting, Inc.",...,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4,"DUNSON HARVESTING, INC",True
7,H-300-20233-780540,Determination Issued - Certification (Expired),2020-09-04 14:26:00.270,2020-10-01 00:00:00.000,Association - Joint Employer,N,Seasonal,N,"FARM LABOR ASSOCIATION FOR GROWERS, INC.",NaN,...,Y,13.50,13.50,55.0,17605922256,flag@sfcos.com,NaN,9,"FARM LABOR ASSOCIATION FOR GROWERS, INC",True
14,H-300-20240-791807,Determination Issued - Certification (Expired),2020-09-17 18:57:56.030,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"MCLAIN FARMS, INC.",NaN,...,N,NaN,12.68,55.0,19125268436,kim@mclainfarms.com,NaN,1,"MCLAIN FARMS, INC",True
17,H-300-20244-795767,Determination Issued - Certification,2020-09-16 12:58:58.600,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"BONNIE PLANTS, INC.",NaN,...,N,NaN,12.68,55.0,13343911328,dennis.ucles@bonnieplants.com,NaN,0,"BONNIE PLANTS, INC",True
18,H-300-20245-799651,Determination Issued - Certification,2020-09-02 11:17:48.730,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"B & W QUALITY GROWERS, INC.",NaN,...,N,NaN,12.68,55.0,17725711135,NaN,www.employflorida.com,7,"B & W QUALITY GROWERS, INC",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2700,H-300-20343-941481,Determination Issued - Certification,2020-12-08 17:59:46.130,2020-12-31 00:00:00.000,Association - Agent,N,Seasonal,N,HARRAL LIVESTOCK CO. LLC,NaN,...,N,NaN,12.68,55.0,13074722105,info@mpaswy.com,NaN,0,HARRAL LIVESTOCK CO LLC,True
2701,H-300-20343-941564,Determination Issued - Certification,2020-12-08 18:13:56.740,2020-12-31 00:00:00.000,Association - Agent,N,Seasonal,N,ECOSYSTEM CONCEPTS INC.,NaN,...,N,NaN,12.68,55.0,13074722105,info@mpaswy.com,NaN,0,ECOSYSTEM CONCEPTS INC,True
2702,H-300-20343-941635,Determination Issued - Certification,2020-12-08 18:27:10.400,2020-12-31 00:00:00.000,Association - Agent,N,Seasonal,N,SIDDOWAY SHEEP CO.,NaN,...,N,NaN,12.68,55.0,13074722105,info@mpaswy.com,NaN,0,SIDDOWAY SHEEP CO,True
2705,H-300-20344-946960,Determination Issued - Certification,2020-12-14 18:01:04.100,2020-12-31 00:00:00.000,Association - Agent,N,Seasonal,N,SATHER MANAGEMENT INC.,NaN,...,Y,12.68,12.68,55.0,15094705742,alfavictor4@yahoo.com,www.WorkSoureWA.com,0,SATHER MANAGEMENT INC,True


In [192]:
jobs_banned_no_punctuation.head()
debar_changed.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,name_clean,changed_name
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"DUNSON HARVESTING, INC.","Dunson Harvesting, Inc.",...,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4,"DUNSON HARVESTING, INC",True
7,H-300-20233-780540,Determination Issued - Certification (Expired),2020-09-04 14:26:00.270,2020-10-01 00:00:00.000,Association - Joint Employer,N,Seasonal,N,"FARM LABOR ASSOCIATION FOR GROWERS, INC.",NaN,...,Y,13.5,13.50,55.0,17605922256,flag@sfcos.com,NaN,9,"FARM LABOR ASSOCIATION FOR GROWERS, INC",True
14,H-300-20240-791807,Determination Issued - Certification (Expired),2020-09-17 18:57:56.030,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"MCLAIN FARMS, INC.",NaN,...,N,NaN,12.68,55.0,19125268436,kim@mclainfarms.com,NaN,1,"MCLAIN FARMS, INC",True
17,H-300-20244-795767,Determination Issued - Certification,2020-09-16 12:58:58.600,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"BONNIE PLANTS, INC.",NaN,...,N,NaN,12.68,55.0,13343911328,dennis.ucles@bonnieplants.com,NaN,0,"BONNIE PLANTS, INC",True
18,H-300-20245-799651,Determination Issued - Certification,2020-09-02 11:17:48.730,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,"B & W QUALITY GROWERS, INC.",NaN,...,N,NaN,12.68,55.0,17725711135,NaN,www.employflorida.com,7,"B & W QUALITY GROWERS, INC",True


,Name,"City, State",Violation,Duration,Start date,End date,violation_num,is_dup,name_clean,changed_name
19,CISCO PRODUCE INC.,"Cairo, GA",Failure to respond to audit (no response),2 years,12/10/2014,12/9/2016,1.0,False,CISCO PRODUCE INC,True
56,CISCO PRODUCE INC.,"Cairo, GA",Impeding the Audit Process – Non- Response,2 years,12/10/2015,12/9/2017,2.0,False,CISCO PRODUCE INC,True
3,ANTON FERTILIZER INC.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,1.0,NaN,ANTON FERTILIZER INC,True
4,"GREAT PLAINS FLUID SERVICE, INC.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016,1.0,NaN,"GREAT PLAINS FLUID SERVICE, INC",True
5,PROMAX INC.,"Whitewater, KS",Failure to Hire U.S. workers,2 years,5/15/2014,5/14/2016,1.0,NaN,PROMAX INC,True


## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [224]:
inner_clean = jobs_banned.merge(debar_clean, left_on='name_clean', right_on='name_clean', how='inner')
# the inner clean did not result in more employers

slash_pattern = r' LLP|[\.]'

debar_clean["name_clean_2"] = [re.sub(slash_pattern, "", employer) for employer in debar_clean.Name]

debar_clean.loc[20]

left_join = jobs_banned.merge(debar_clean, left_on='name_clean', right_on='name_clean_2', how='left', indicator = True)
left_join[left_join["_merge"]=="both"]


Name             SLASH E.V. RANCH LLP
City, State                 Rifle, CO
Violation               WHD Debarment
Duration                       1 year
Start date                 11/15/2014
End date                   11/14/2015
violation_num                     1.0
is_dup                            NaN
name_clean       SLASH E.V. RANCH LLP
changed_name                    False
name_clean_2           SLASH EV RANCH
Name: 20, dtype: object

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,Violation,Duration,Start date,End date,violation_num,is_dup,name_clean_y,changed_name_y,name_clean_2,_merge
791,H-300-20287-876656,Determination Issued - Certification,2020-10-20 09:20:32.010,2020-11-09 00:00:00.000,Individual Employer,Y,Seasonal,Y,RAFAEL BARAJAS,NaN,...,Non-payment of certification fee,1 year,9/23/2016,9/22/2017,1.0,NaN,RAFAEL BARAJAS,False,RAFAEL BARAJAS,both
1115,H-300-20306-894148,Determination Issued - Certification,2020-11-02 18:11:29.140,2020-11-24 00:00:00.000,Individual Employer,N,Seasonal,N,SLASH EV RANCH,NaN,...,WHD Debarment,1 year,11/15/2014,11/14/2015,1.0,NaN,SLASH E.V. RANCH LLP,False,SLASH EV RANCH,both


# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [ ]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [ ]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [ ]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [ ]:
## your code here